<a href="https://colab.research.google.com/github/guebin/DL2024/blob/main/posts/05wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

{{<video https://youtu.be/playlist?list=PLQqh36zP38-wjNGgd4gmQJbQ66NLjUC2y&si=dusDZAwGOJS9TOKJ >}}

# 2. Imports

In [8]:
import torch
import matplotlib.pyplot as plt
import torchvision
import fastai.data.all 

In [17]:
from torchvision.io import read_image

# 3. CPU vs GPU 

`-` 파이토치에서 GPU를 쓰는 방법을 알아보자. (사실 지금까지 우리는 CPU만 쓰고 있었음) 

`-` 코랩에서 GPU설정하는 방법: (아래영상참고)

{{<video https://youtu.be/GyBL2YBCzoc >}}

## A. GPU 사용방법 

`-` cpu 연산이 가능한 메모리에 데이터 저장

In [2]:
torch.manual_seed(43052)
x_cpu = torch.tensor([0.0,0.1,0.2]).reshape(-1,1) 
y_cpu = torch.tensor([0.0,0.2,0.4]).reshape(-1,1) 
net_cpu = torch.nn.Linear(1,1) 

`-` gpu 연산이 가능한 메모리에 데이터 저장 

In [3]:
!nvidia-smi # before

Tue Aug 20 15:32:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:01:00.0  On |                  N/A |
|  0%   56C    P3              88W / 420W |   1052MiB / 24576MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
torch.manual_seed(43052)
x_gpu = x_cpu.to("cuda:0")
y_gpu = y_cpu.to("cuda:0")
net_gpu = torch.nn.Linear(1,1).to("cuda:0") 

In [5]:
!nvidia-smi

Tue Aug 20 15:33:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:01:00.0  On |                  N/A |
|  0%   58C    P2             135W / 420W |   1321MiB / 24576MiB |     16%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

> GPU에 메모리를 올리면 GPU메모리가 점유된다! (26MiB -> 287MiB) 

`-` cpu 혹은 gpu 연산이 가능한 메모리에 저장된 값들을 확인 

In [6]:
x_cpu, y_cpu, net_cpu.weight, net_cpu.bias

(tensor([[0.0000],
         [0.1000],
         [0.2000]]),
 tensor([[0.0000],
         [0.2000],
         [0.4000]]),
 Parameter containing:
 tensor([[-0.3467]], requires_grad=True),
 Parameter containing:
 tensor([-0.8470], requires_grad=True))

In [7]:
x_gpu, y_gpu, net_gpu.weight, net_gpu.bias

(tensor([[0.0000],
         [0.1000],
         [0.2000]], device='cuda:0'),
 tensor([[0.0000],
         [0.2000],
         [0.4000]], device='cuda:0'),
 Parameter containing:
 tensor([[-0.3467]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-0.8470], device='cuda:0', requires_grad=True))

`-` gpu는 gpu끼리 연산가능하고 cpu는 cpu끼리 연산가능함 

(예시1)

In [8]:
net_cpu(x_cpu) 

tensor([[-0.8470],
        [-0.8817],
        [-0.9164]], grad_fn=<AddmmBackward0>)

(예시2)

In [9]:
net_gpu(x_gpu) 

tensor([[-0.8470],
        [-0.8817],
        [-0.9164]], device='cuda:0', grad_fn=<AddmmBackward0>)

(예시3)

In [ ]:
net_cpu(x_gpu) 

In [ ]:
x_cpu.to("cuda:0")
net_gpu(x_cpu) 

In [ ]:
net_gpu(x_cpu.to("cuda:0")
) 

::: {.callout-note}
#### 강의중 net을 재선언한 이유

`-` 아래와 같이 `x_cpu` 혹은 `y_cpu`에 `.to("cuda:0")`메소드를 쓸 경우 

```Python
x_cpu.to("cuda:0")
y_cpu.to("cuda:0")
```

`x_cpu`와 `y_cpu`는 cpu에 그대로 있음. 

`-` 그런데 아래와 같이 `net_cpu`에서 `.to("cuda:0")`메소드를 쓸 경우 

```Python
net_cpu.to("cuda:0")
```

`net_cpu` 자체가 gpu에 올라가게 됨. 
:::

(예시4)

(예시5)

(예시6)

In [10]:
torch.mean((y_gpu-net_gpu(x_gpu))**2)

tensor(1.2068, device='cuda:0', grad_fn=<MeanBackward0>)

(예시7)

In [11]:
torch.mean((y_gpu-net_cpu(x_cpu))**2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

(예시8)

In [12]:
torch.mean((y_cpu-net_gpu(x_gpu))**2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [13]:
net_gpu(x_cpu)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
torch.mean((y_cpu-net_cpu(x_cpu))**2)

## B. 시간측정 (예비학습) 

In [14]:
import time 

In [15]:
t1 = time.time()

In [16]:
t2 = time.time()

In [17]:
t2-t1

0.30453991889953613

## C. CPU vs GPU (512 nodes)

`-` CPU (512 nodes) 

In [18]:
torch.manual_seed(5) 
x=torch.linspace(0,1,100).reshape(-1,1)
y=torch.randn(100).reshape(-1,1)*0.01
#---#
net = torch.nn.Sequential(
    torch.nn.Linear(1,512),
    torch.nn.ReLU(),
    torch.nn.Linear(512,1)
)
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
t1 = time.time()
for epoc in range(1000):
    # 1 
    yhat = net(x)
    # 2 
    loss = loss_fn(yhat,y)
    # 3 
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()
t2 = time.time()
t2-t1

0.3479437828063965

`-` GPU (512 nodes) 

In [ ]:
torch.manual_seed(5) 
x=torch.linspace(0,1,100).reshape(-1,1).to("cuda:0")
y=(torch.randn(100).reshape(-1,1)*0.01).to("cuda:0")
#---#
net = torch.nn.Sequential(
    torch.nn.Linear(1,512),
    torch.nn.ReLU(),
    torch.nn.Linear(512,1)
).to("cuda:0")
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
t1 = time.time()
for epoc in range(1000):
    # 1 
    yhat = net(x)
    # 2 
    loss = loss_fn(yhat,y)
    # 3 
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()
t2 = time.time()
t2-t1

- CPU가 더 빠르다??

## D. CPU vs GPU (20,480 nodes)

`-` CPU (20,480)

In [ ]:
torch.manual_seed(5) 
x=torch.linspace(0,1,100).reshape(-1,1)
y=torch.randn(100).reshape(-1,1)*0.01
#---#
net = torch.nn.Sequential(
    torch.nn.Linear(1,20480),
    torch.nn.ReLU(),
    torch.nn.Linear(20480,1)
)
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
t1 = time.time()
for epoc in range(1000):
    # 1 
    yhat = net(x)
    # 2 
    loss = loss_fn(yhat,y)
    # 3 
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()
t2 = time.time()
t2-t1

`-` GPU (20,480)

In [ ]:
torch.manual_seed(5) 
x=torch.linspace(0,1,100).reshape(-1,1).to("cuda:0")
y=(torch.randn(100).reshape(-1,1)*0.01).to("cuda:0")
#---#
net = torch.nn.Sequential(
    torch.nn.Linear(1,20480),
    torch.nn.ReLU(),
    torch.nn.Linear(20480,1)
).to("cuda:0")
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
t1 = time.time()
for epoc in range(1000):
    # 1 
    yhat = net(x)
    # 2 
    loss = loss_fn(yhat,y)
    # 3 
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()
t2 = time.time()
t2-t1

- 왜 이런 차이가 나는가? 
- 연산을 하는 주체는 코어인데 CPU는 수는 적지만 일을 잘하는 코어들을 가지고 있고 GPU는 일은 못하지만 다수의 코어를 가지고 있기 때문 

## E. CPU vs GPU (204,800 nodes)

`-` CPU (204,800)

In [ ]:
torch.manual_seed(5) 
x=torch.linspace(0,1,100).reshape(-1,1)
y=torch.randn(100).reshape(-1,1)*0.01
#---#
net = torch.nn.Sequential(
    torch.nn.Linear(1,204800),
    torch.nn.ReLU(),
    torch.nn.Linear(204800,1)
)
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
t1 = time.time()
for epoc in range(1000):
    # 1 
    yhat = net(x)
    # 2 
    loss = loss_fn(yhat,y)
    # 3 
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()
t2 = time.time()
t2-t1

`-` GPU (204,800)

In [ ]:
torch.manual_seed(5) 
x=torch.linspace(0,1,100).reshape(-1,1).to("cuda:0")
y=(torch.randn(100).reshape(-1,1)*0.01).to("cuda:0")
#---#
net = torch.nn.Sequential(
    torch.nn.Linear(1,204800),
    torch.nn.ReLU(),
    torch.nn.Linear(204800,1)
).to("cuda:0")
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
t1 = time.time()
for epoc in range(1000):
    # 1 
    yhat = net(x)
    # 2 
    loss = loss_fn(yhat,y)
    # 3 
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()
t2 = time.time()
t2-t1

# 4. "확률적" 경사하강법

## A. 의문: 좀 이상하지 않아요? 

`-` 국민상식: GPU 비싸요.. <https://bbs.ruliweb.com/community/board/300143/read/61066881>

- GPU 메모리 많아봐야 24GB, 그래도 비싸요.. <http://shop.danawa.com/virtualestimate/?controller=estimateMain&methods=index&marketPlaceSeq=16>
- GPU 메모리가 80GB일 경우 가격: <https://prod.danawa.com/info/?pcode=21458333>

`-` 우리가 분석하는 데이터: 빅데이터..?

In [ ]:
x = torch.linspace(-10,10,100000).reshape(-1,1)
eps = torch.randn(100000).reshape(-1,1)
y = x*2 + eps 

In [ ]:
plt.plot(x,y,'o',alpha=0.05)
plt.plot(x,2*x,'--')

`-` 데이터의 크기가 커지는 순간 `X.to("cuda:0")`, `y.to("cuda:0")` 쓰면 난리나겠는걸? 

`-` 데이터를 100개중에 1개만 꼴로만 쓰면 어떨까? 

In [ ]:
plt.plot(x[::100],y[::100],'o',alpha=0.05)
plt.plot(x,2*x,'--')

In [ ]:
x[::1000].shape # 1000개씩 점프하면서

- 대충 이거만 가지고 적합해도 충분히 정확할것 같은데?

## B. X,y 데이터를 굳이 모두 GPU에 넘겨야 하는가?

`-` 데이터셋을 짝홀로 나누어서 번갈아가면서 GPU에 올렸다 내렸다하면 안되나? 

`-` 아래의 알고리즘을 생각해보자. 

1. 데이터를 반으로 나눈다. 
2. 짝수obs의 x,y 그리고 net의 모든 파라메터를 GPU에 올린다. 
3. yhat, loss, grad, update 수행
4. 짝수obs의 x,y를 GPU메모리에서 내린다. 그리고 홀수obs의 x,y를 GPU메모리에 올린다. 
5. yhat, loss, grad, update 수행 
6. 홀수obs의 x,y를 GPU메모리에서 내린다. 그리고 짝수obs의 x,y를 GPU메모리에 올린다. 
7. 반복 

> 이러면 되는거아니야???? ---> 맞아요

## C. 경사하강법, 확률적경사하강법, 미니배치 경사하강법 

10개의 샘플이 있다고 가정. $\{(x_i,y_i)\}_{i=1}^{10}$

`# ver1` --  모든 샘플을 이용하여 slope 계산

(epoch 1) $loss=\sum_{i=1}^{10}(y_i-w_0-w_1x_i)^2 \to slope  \to update$

(epoch 2) $loss=\sum_{i=1}^{10}(y_i-w_0-w_1x_i)^2 \to slope  \to update$

...

> 우리가 항상 이렇게 했죠!

`# ver2` -- 하나의 샘플만을 이용하여 slope 계산

(epoch 1) 

- $loss=(y_1-w_0-w_1x_1)^2 \to slope \to update$
- $loss=(y_2-w_0-w_1x_2)^2 \to slope \to update$
- ...
- $loss=(y_{10}-w_0-w_1x_{10})^2  \to  slope  \to  update$

(epoch 2) 

- $loss=(y_1-w_0-w_1x_1)^2  \to slope  \to  update$
- $loss=(y_2-w_0-w_1x_2)^2  \to slope  \to  update$
- ...
- $loss=(y_{10}-w_0-w_1x_{10})^2  \to  slope  \to  update$

...

`# ver3` -- $m (\leq n)$ 개의 샘플을 이용하여 slope 계산

$m=3$이라고 하자. 

(epoch 1) 

- $loss=\sum_{i=1}^{3}(y_i-w_0-w_1x_i)^2  \to  slope  \to  update$
- $loss=\sum_{i=4}^{6}(y_i-w_0-w_1x_i)^2  \to  slope  \to  update$
- $loss=\sum_{i=7}^{9}(y_i-w_0-w_1x_i)^2  \to  slope  \to  update$
- $loss=(y_{10}-w_0-w_1x_{10})^2  \to  slope  \to  update$

(epoch 2) 

- $loss=\sum_{i=1}^{3}(y_i-w_0-w_1x_i)^2  \to  slope  \to  update$
- $loss=\sum_{i=4}^{6}(y_i-w_0-w_1x_i)^2  \to  slope  \to  update$
- $loss=\sum_{i=7}^{9}(y_i-w_0-w_1x_i)^2  \to  slope  \to  update$
- $loss=(y_{10}-w_0-w_1x_{10})^2  \to  slope  \to  update$

...

## D. 용어의 정리 

**옛날**

`-` ver1: gradient descent, batch gradient descent

`-` ver2: stochastic gradient descent (sample 1개)

`-` ver3: mini-batch gradient descent, mini-batch stochastic gradient descent (sample n개 == batch)

**요즘**

`-` ver1: gradient descent

`-` ver2: stochastic gradient descent with batch size = 1

`-` **ver3: stochastic gradient descent**
- https://www.deeplearningbook.org/contents/optimization.html, 알고리즘 8-1 참고.

## E. Dataset(`ds`), DataLoader(`dl`)

> 취지는 알겠으나, C의 과정을 실제 구현하려면 진짜 힘들것 같아요.. (입코딩과 손코딩의 차이) --> 이걸 해결하기 위해서 파이토치에서는 DataLoader라는 오브젝트를 준비했음!

`-` ds: 섭스크립터블함

In [19]:
x=torch.tensor(range(10)).float().reshape(-1,1)
y=torch.tensor([1.0]*5+[0.0]*5).reshape(-1,1)
torch.concat([x,y],axis=1)

tensor([[0., 1.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [5., 0.],
        [6., 0.],
        [7., 0.],
        [8., 0.],
        [9., 0.]])

In [20]:
torch.concat([x,y],axis=1).shape

torch.Size([10, 2])

In [21]:
torch.utils.data.TensorDataset?

Init signature: torch.utils.data.TensorDataset(*args, **kwds)
Docstring:     
Dataset wrapping tensors.

Each sample will be retrieved by indexing tensors along the first dimension.

Args:
    *tensors (Tensor): tensors that have the same size of the first dimension.
File:           ~/.conda/envs/dl2024/lib/python3.8/site-packages/torch/utils/data/dataset.py
Type:           type
Subclasses:     

In [22]:
# dataset
ds=torch.utils.data.TensorDataset(x,y)
ds  # 뭐 정보가 없네.. 어쩌란거지..?

In [23]:
dir(ds)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'tensors']

In [24]:
len(ds)

10

In [25]:
set(dir(ds)) & {'__iter__'}


set()

In [26]:
set(dir(ds)) & {'__getitem__'}


{'__getitem__'}

__iter__ 이외에도 __getitem__ 이 있으면 for 문을 사용 가능하다.

In [29]:
for i in ds:
    print(i)

(tensor([0.]), tensor([1.]))
(tensor([1.]), tensor([1.]))
(tensor([2.]), tensor([1.]))
(tensor([3.]), tensor([1.]))
(tensor([4.]), tensor([1.]))
(tensor([5.]), tensor([0.]))
(tensor([6.]), tensor([0.]))
(tensor([7.]), tensor([0.]))
(tensor([8.]), tensor([0.]))
(tensor([9.]), tensor([0.]))


In [35]:
for i in ds:
    print(i[0])

tensor([0.])
tensor([1.])
tensor([2.])
tensor([3.])
tensor([4.])
tensor([5.])
tensor([6.])
tensor([7.])
tensor([8.])
tensor([9.])


In [31]:
iter(ds)

In [39]:
index = 0

while True:
    try:
        # __getitem__ 메서드를 사용하여 인덱스를 통해 요소에 접근
        # print(index)
        item = ds.__getitem__(index)
        print(item)
        index += 1
    except IndexError:
        # 더 이상 요소가 없으면 IndexError가 발생하므로 반복을 종료
        break

(tensor([0.]), tensor([1.]))
(tensor([1.]), tensor([1.]))
(tensor([2.]), tensor([1.]))
(tensor([3.]), tensor([1.]))
(tensor([4.]), tensor([1.]))
(tensor([5.]), tensor([0.]))
(tensor([6.]), tensor([0.]))
(tensor([7.]), tensor([0.]))
(tensor([8.]), tensor([0.]))
(tensor([9.]), tensor([0.]))


In [41]:
#set(dir(ds)) & {'__getitem__'}
# __getitem__ 이 있다는건 리스트처럼 사용하는게 가능하다는 것! , 추가로 for 문도 가능함.
ds[0] ,ds[1] , ds[7]

((tensor([0.]), tensor([1.])),
 (tensor([1.]), tensor([1.])),
 (tensor([7.]), tensor([0.])))

In [42]:
set(dir(ds)) & {'tensors'}


{'tensors'}

In [54]:
import numpy as np

In [55]:
ds.tensors 

(tensor([[0.],
         [1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [7.],
         [8.],
         [9.]]),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]))

In [64]:
_a,_b=ds.tensors 
b_numpy = _b.numpy()


In [68]:
np.array(b_numpy).shape

(10, 1)

In [69]:
set(dir(ds.tensors)) & {'__getitem__'}

{'__getitem__'}

In [70]:
# __getitem__ 이 있으면 List 처럼 사용가능하기도 하고
_aa,_bb= ds.tensors 
_aa


tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [71]:
set(dir(_aa)) & {'__getitem__'}

{'__getitem__'}

In [96]:
# __getitem__ 이 있으면 for문에서  사용가능하기도 한다.
for i in _aa:
    print(i.tolist())
    print(i[0].tolist())
    

[0.0]
0.0
[1.0]
1.0
[2.0]
2.0
[3.0]
3.0
[4.0]
4.0
[5.0]
5.0
[6.0]
6.0
[7.0]
7.0
[8.0]
8.0
[9.0]
9.0


In [78]:
ds[0],(x,y)[0] # (x,y) 튜플자체는 아님.. 인덱싱이 다르게 동작

((tensor([0.]), tensor([1.])),
 tensor([[0.],
         [1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [7.],
         [8.],
         [9.]]))

In [79]:
(x,y)[1]

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

::: {.callout-note}
여기서 제가 `__iter__` 가 숨겨져 있는 오브젝트일 경우만 `for`문이 동작한다고 설명 했는데요, `__getitem__`이 있는 경우도 동작한다고 합니다. **제가 잘못 알고 있었어요. 혼란을 드려 죄송합니다**. 

- 그래도 `dl`은 for 를 돌리기위해서 만든 오브젝트라는 설명은 맞는 설명입니다. 
- `ds`역시 독특한 방식의 인덱싱을 지원하도록 한 오브젝트라는 설명도 맞는 설명입니다. 
:::

`-` dl: 섭스크립터블하지 않지만 이터러블함 
- __getitem__ 이 없어서 섭스크립터블하지 않지만
- __iter__ 이 있어서 이터러블하긴 함

In [ ]:
torch.utils.data.DataLoader?

In [81]:
## dataset
# ds=torch.utils.data.TensorDataset(x,y)

# dataloader
dl=torch.utils.data.DataLoader(dataset=ds,batch_size=3)
# dl = torch.utils.data.DataLoader(dataset=ds, batch_size=64, shuffle=True, num_workers=4)

#set(dir(dl)) & {'__iter__'}

In [84]:
set(dir(dl)) & {'__getitem__'}


set()

__iter__ 가 있으니 for 문을 돌릴 수 있다.

In [85]:
set(dir(dl)) & {'__iter__'}


{'__iter__'}

In [86]:
for i in dl:
    print(i)

[tensor([[0.],
        [1.],
        [2.]]), tensor([[1.],
        [1.],
        [1.]])]
[tensor([[3.],
        [4.],
        [5.]]), tensor([[1.],
        [1.],
        [0.]])]
[tensor([[6.],
        [7.],
        [8.]]), tensor([[0.],
        [0.],
        [0.]])]
[tensor([[9.]]), tensor([[0.]])]


In [87]:
for xi,yi in dl:
    # print(f"x_batch:{xi} \t y_batch:{yi}")
    print(type(xi))
    print(type(yi))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [88]:
for xi,yi in dl:
    print(f"x_batch:{xi} \t y_batch:{yi}")

x_batch:tensor([[0.],
        [1.],
        [2.]]) 	 y_batch:tensor([[1.],
        [1.],
        [1.]])
x_batch:tensor([[3.],
        [4.],
        [5.]]) 	 y_batch:tensor([[1.],
        [1.],
        [0.]])
x_batch:tensor([[6.],
        [7.],
        [8.]]) 	 y_batch:tensor([[0.],
        [0.],
        [0.]])
x_batch:tensor([[9.]]) 	 y_batch:tensor([[0.]])


In [89]:
for xi,yi in dl:
    print(f"x_batch:{xi.tolist()} \t y_batch:{yi.tolist()}")

x_batch:[[0.0], [1.0], [2.0]] 	 y_batch:[[1.0], [1.0], [1.0]]
x_batch:[[3.0], [4.0], [5.0]] 	 y_batch:[[1.0], [1.0], [0.0]]
x_batch:[[6.0], [7.0], [8.0]] 	 y_batch:[[0.0], [0.0], [0.0]]
x_batch:[[9.0]] 	 y_batch:[[0.0]]


In [97]:
len(dl) 

4

`-` 마지막관측치는 뭔데 단독으로 업데이트하냐?? 편향되는 위험이 있는거 아냐?? --> shuffle True 같이 자잘한 옵션도 있음.. 

In [98]:
dl = torch.utils.data.DataLoader(ds,batch_size=3,shuffle=True)
for xi,yi in dl:
    print(f'x_batch={xi.tolist()} \t y_batch={yi.tolist()}')

x_batch=[[9.0], [2.0], [6.0]] 	 y_batch=[[0.0], [1.0], [0.0]]
x_batch=[[0.0], [8.0], [4.0]] 	 y_batch=[[1.0], [0.0], [1.0]]
x_batch=[[1.0], [7.0], [5.0]] 	 y_batch=[[1.0], [0.0], [0.0]]
x_batch=[[3.0]] 	 y_batch=[[1.0]]


`-` DataLoader의 주요 파라미터
- dataset: 로드할 데이터셋. Dataset 클래스의 인스턴스
- batch_size: 배치 크기. 한 번에 로드할 데이터 포인트의 수
- shuffle: 매 에폭(epoch)마다 데이터를 섞을지 여부.
    - epoch: 전체 데이터셋이 학습 알고리즘을 통과하는 한 번의 완전한 사이클을 의미
    - 일반적으로 학습 데이터셋에는 True를, 검증/테스트 데이터셋에는 False를 사용
- num_workers: 데이터 로딩에 사용할 서브 프로세스의 수. 더 빠른 데이터 로딩을 위해 0보다 큰 값을 설정할 수 있습니다.
    - 이 값이 0이면, 메인 프로세스가 모든 데이터 로딩 작업을 수행합니다.
- collate_fn: 배치 데이터를 어떻게 처리할지 정의하는 함수입니다. 사용자 정의 데이터 처리를 위해 커스터마이즈할 수 있습니다.

## F. ds, dl을 이용한 MNIST 구현

`-` 목표: 확률적경사하강법과 그냥 경사하강법의 성능을 "동일 반복횟수"로 비교해보자.

- batch_size = 2048로 설정할것 

`-` 그냥 경사하강법 -- 미니배치 안쓰는 학습, 우리가 맨날하는 그거

In [10]:
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')

In [11]:
path

Path('/home/myuser/.fastai/data/mnist_png')

In [13]:
torchvision.io.read_image(path/'training/0/1775.png').shape

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

In [5]:
!conda install pytorch torchvisio -c conda

Retrieving notices: ...working... done
Channels:
 - conda
 - defaults
 - conda-forge
 - nvidia
 - fastai
 - pytorch
Platform: linux-64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - torchvisio

Current channels:

  - https://conda.anaconda.org/conda
  - defaults
  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/nvidia/linux-64
  - https://conda.anaconda.org/fastai/noarch
  - https://conda.anaconda.org/pytorch/linux-64
  - https://conda.anaconda.org/pytorch/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [9]:
plt.imshow(torchvision.io.read_image(path/'training/0/1775.png').reshape(28,28),cmap="gray")

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

In [108]:
aaa=([torchvision.io.read_image(str(i)) for i in (path/'training/3').ls()])
aaa[0].shape , aaa[1].shape 

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

In [109]:
len(aaa)

NameError: name 'aaa' is not defined

In [ ]:
aaa[0].shape

In [117]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
# X0 train 데이터셋
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
# X1 train 데이터셋
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
print(f'X0.shape={X0.shape} \t X1.shape={X1.shape}')
# X0 와 X1 을 합친 X train 데이터셋
X = torch.concat([X0,X1],axis=0).reshape(-1,1*28*28)/255    # int 에서 float로 , shape 는 12665,784
print(f'X.shape={X.shape}')

y = torch.tensor([0.0]*len(X0) + [1.0]*len(X1)).reshape(-1,1) # shape 는 12665,1
print(f'y.shape={y.shape}')

## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(1)
net = torch.nn.Sequential(
    # 중간 노드 32 
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    # 최종 출력노드 1
    torch.nn.Linear(32,1),
    # 이진분류 Sigmoid
    torch.nn.Sigmoid()
)
# 이진분류 Loss
loss_fn = torch.nn.BCELoss()
optimizr = torch.optim.SGD(net.parameters())        # Stochastic Gradient Descent 를 테스트 중이니 Adam은 사용하지 말자. 여기선 batch=data_size_all
## Step3: fit  
for epoc in range(700):     # 여기선 epoch 을 700 으로
    # step1 
    yhat = net(X)
    # step2 
    loss = loss_fn(yhat,y)
    # step3     
    loss.backward()
    # step4 
    optimizr.step()
    optimizr.zero_grad()
## Step4: Predict 
((yhat > 0.5)*1.0 ==  y).float().mean()

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

In [ ]:
net(X[0].reshape(-1,784))

In [ ]:
net(X[0]).reshape(-1,28*28)

In [ ]:
plt.imshow(X0[0].reshape(28,28),cmap="gray")

In [ ]:
X[0].shape    # int 에서 float로

In [ ]:
# dl 다시 복습
dl = torch.utils.data.DataLoader(ds,batch_size=3)
for xi,yi in dl:
    print(f'x_batch={xi.tolist()} \t y_batch={yi.tolist()}')

In [ ]:
len(dl)

`-` "확률적"경사하강법 -- 미니배치 쓰는 학습 

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X = torch.concat([X0,X1],axis=0).reshape(-1,1*28*28)/255
y = torch.tensor([0.0]*len(X0) + [1.0]*len(X1)).reshape(-1,1)

# batch 를 사용하기위해 Dataset과 Dataloader 생성
ds = torch.utils.data.TensorDataset(X,y)
# batch size = 2048
dl = torch.utils.data.DataLoader(ds,batch_size=2048)
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(1)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
    torch.nn.Sigmoid()
)
loss_fn = torch.nn.BCELoss()
optimizr = torch.optim.SGD(net.parameters())        # Stochastic Gradient Descent 를 테스트 중이니 Adam은 사용하지 말자. 여기선 batch=2048
# ## Step3: fit  
for epoc in range(100):     # 여기선 epoch 을 100 으로
    for xi,yi in dl:        # len(dl) == 7    , 데이터 수 / batch_size 하면 6.xx 나오기때문.
        # step1 
        #yihat = net(xi)
        # step2 
        loss = loss_fn(net(xi),yi)
        # step3     
        loss.backward()
        # step4 
        optimizr.step()
        optimizr.zero_grad()
# ## Step4: Predict 
((net(X) > 0.5)*1.0 ==  y).float().mean()

In [ ]:
len(dl)

In [ ]:
X0.shape

In [ ]:
for xi,yi in dl:
    print(f'x_batch={len(xi.tolist())} \t y_batch={len(yi.tolist())}')

`-` GPU를 활용하는 "확률적"경사하강법 -- 실제적으로는 이게 최종알고리즘 

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X = torch.concat([X0,X1],axis=0).reshape(-1,1*28*28)/255
y = torch.tensor([0.0]*len(X0) + [1.0]*len(X1)).reshape(-1,1)

# batch 를 사용하기위해 Dataset과 Dataloader 생성
ds = torch.utils.data.TensorDataset(X,y)
dl = torch.utils.data.DataLoader(ds,batch_size=2048)
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(1)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
    torch.nn.Sigmoid()
).to("cuda:0")
loss_fn = torch.nn.BCELoss()
optimizr = torch.optim.SGD(net.parameters())
## Step3: fit  
for epoc in range(100):     # 여기선 epoch 을 100 으로
    for xi,yi in dl:        
        # step1 
        # step2 
        loss = loss_fn(net(xi.to("cuda:0")),yi.to("cuda:0"))
        # step3     
        loss.backward()
        # step4 
        optimizr.step()
        optimizr.zero_grad()
# ## Step4: Predict
net.to("cpu")
((net(X) > 0.5)*1.0 ==  y).float().mean()

In [ ]:
type([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])

In [ ]:
[torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()][0].shape

In [ ]:
X0.shape

In [ ]:
X.shape

In [ ]:
2048*6 + 377

# 5. 다중클래스 분류

## A. 결론 (그냥 외우세요)

`-` 2개의 class를 구분하는 문제가 아니라 $k$개의 class를 구분해야 한다면? 

***일반적인 개념*** 

- 손실함수: BCE loss $\to$ Cross Entropy loss 
- 마지막층의 선형변환: torch.nn.Linear(?,1) $\to$ torch.nn.Linear(?,k) 
- **마지막층의 활성화: sig $\to$ softmax**

***파이토치 한정*** 

- **y의형태: (n,) vector + int형 // (n,k) one-hot encoded matrix + float형**
- 손실함수: torch.nn.BCEWithLogitsLoss,  $\to$ torch.nn.CrossEntropyLoss
- 마지막층의 선형변환: torch.nn.Linear(?,1) $\to$ torch.nn.Linear(?,k) 
- **마지막층의 활성화: None $\to$ None (손실함수에 이미 마지막층의 활성화가 포함)**

## B. 실습: 3개의 클래스를 구분 

`-` 정리된 코드1: 통계잘하는데 파이토치 못쓰는 사람의 코드
- **y의형태 : one-hot encoded matrix + float형**  

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X2 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/2').ls()])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)/255
y = torch.nn.functional.one_hot(torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2))).float()
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,3),
#    torch.nn.Softmax()
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
## Step3: 적합 
for epoc in range(100):
    ## step1 
    netout = net(X)
    ## step2 
    loss = loss_fn(netout,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()
    
## Step4: 적합 (혹은 적합결과확인)
(netout.argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [ ]:
#[(str(fname)) for fname in (path/'training/0').ls()]

In [ ]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()


In [ ]:
net(X[1]), plt.imshow(X[0].reshape(28,28)), y[0]

In [ ]:
net(X[1]), plt.imshow(X[-1].reshape(28,28)), y[-1]

`-` 정리된 코드2: 파이토치를 잘하는 사람의 코드 :
- **y의형태: (n,) vector + int형**


In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X2 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/2').ls()])
X = torch.concat([X0,X1,X2]).reshape(-1,1*28*28)/255
#y = torch.nn.functional.one_hot(torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2))).float()
y = torch.tensor([0]*len(X0) + [1]*len(X1)+ [2]*len(X2))    # int 타입
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,3),
#    torch.nn.Softmax()
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
## Step3: 적합 
for epoc in range(100):
    ## step1 
    netout = net(X)
    ## step2 
    loss = loss_fn(netout,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()
## Step4: 적합 (혹은 적합결과확인)    
(netout.argmax(axis=1) == y).float().mean()

In [ ]:
y.dtype , y.shape

- 완전같은코드임 

## C. Softmax 

`-` 눈치: softmax를 쓰기 직전의 숫자들은 (n,k)꼴로 되어있음. 각 observation 마다 k개의 숫자가 있는데, 그중에서 유난히 큰 하나의 숫자가 있음. 

In [ ]:
net(X)

In [ ]:
y

`-` 수식 

- $\text{sig}(u)=\frac{e^u}{1+e^u}$
- $\text{softmax}({\boldsymbol u})=\text{softmax}([u_1,u_2,\dots,u_k])=\big[ \frac{e^{u_1}}{e^{u_1}+\dots e^{u_k}},\dots,\frac{e^{u_k}}{e^{u_1}+\dots e^{u_k}}\big]$

`-` torch.nn.Softmax() 손계산 

(예시1) -- 잘못계산 : dim 이 잘못됨

In [ ]:
softmax = torch.nn.Softmax(dim=0)

In [ ]:
# 데이터의 개수 : 5개
netout = torch.tensor([[-2.0,-2.0,0.0],
                        [3.14,3.14,3.14],
                        [0.0,0.0,2.0],
                        [2.0,2.0,4.0],
                        [0.0,0.0,0.0]])
netout

In [ ]:
softmax(netout) 

(예시2) -- 이게 맞게 계산되는 것임 

In [ ]:
softmax = torch.nn.Softmax(dim=1)

In [ ]:
netout

In [ ]:
softmax(netout)

(예시3) -- 차원을 명시안하면 맞게 계산해주고 경고 줌 

In [ ]:
softmax = torch.nn.Softmax()

In [ ]:
softmax = torch.nn.Softmax?

In [ ]:
netout

In [ ]:
softmax(netout)

(예시4) -- 진짜 손계산 

In [ ]:
netout 

In [ ]:
torch.exp(netout)

In [ ]:
0.1353/(0.1353 + 0.1353 + 1.0000), 0.1353/(0.1353 + 0.1353 + 1.0000), 1.0000/(0.1353 + 0.1353 + 1.0000) # 첫 obs

In [ ]:
torch.exp(netout[1])/torch.exp(netout[1]).sum() # 두번째 obs 

In [ ]:
softmax(netout)[1]

## D. CrossEntropyLoss

`-` 수식 

***`# 2개의 카테고리`***

`-` 예제1: BCELoss vs BCEWithLogisticLoss

In [ ]:
y = torch.tensor([0,0,1]).reshape(-1,1).float()
netout = torch.tensor([-1, 0, 1]).reshape(-1,1).float()
y,netout

In [ ]:
# 계산방법1: 공식암기
sig = torch.nn.Sigmoid()
yhat = sig(netout)


In [ ]:
- torch.sum(torch.log(yhat)*y + torch.log(1-yhat)*(1-y))/3      # BCE Loss 손으로 구하기

In [ ]:
yhat , yhat.sum()==1

In [ ]:
y , y.sum()==1

In [ ]:
# 계산방법2: torch.nn.BCELoss() 이용
sig = torch.nn.Sigmoid()
yhat = sig(netout)
loss_fn = torch.nn.BCELoss()
loss_fn(yhat,y)

In [ ]:
# 계산방법3: torch.nn.BCEWithLogitsLoss() 이용
loss_fn = torch.nn.BCEWithLogitsLoss()
loss_fn(netout,y)

`-` 예제2: BCEWithLogisticLoss vs CrossEntropyLoss

In [ ]:
torch.concat([sig(netout),1-sig(netout)],axis=1)

In [ ]:
# 데이터의 개수 : 3개
netout = torch.tensor([[3,2],[2,2],[5,6]]).float()
y = torch.tensor([[1,0],[1,0],[0,1]]).float()
y,netout #,netout[:,[1]]-netout[:,[0]]

In [ ]:
netout.shape

In [ ]:
softmax(netout)

In [ ]:
softmax(netout)*y

In [ ]:
# 계산방법1: 공식암기
-torch.sum(torch.log(softmax(netout))*y)/3

In [ ]:
# 계산방법2: torch.nn.CrossEntropyLoss() 이용 + y는 one-hot으로 정리
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y)

In [ ]:
# 계산방법3: torch.nn.CrossEntropyLoss() 이용 + y는 0,1 로 정리
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y)

`#`

***`# 3개의 카테고리, 데이터의 개수는 5개`***

In [ ]:
y = torch.tensor([2,1,2,2,0])
y_onehot = torch.nn.functional.one_hot(y)
netout = torch.tensor(
    [[-2.0000, -2.0000,  0.0000],
     [ 3.1400,  3.1400,  3.1400],
     [ 0.0000,  0.0000,  2.0000],
     [ 2.0000,  2.0000,  4.0000],
     [ 0.0000,  0.0000,  0.0000]]
)
y,y_onehot

In [ ]:
## 방법1 -- 추천X
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y_onehot.float())

In [ ]:
## 방법2 -- 추천O
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(netout,y)

In [ ]:
y.dtype

In [ ]:
## 방법3 -- 공식.. (이걸 쓰는사람은 없겠지?)
softmax = torch.nn.Softmax() 
loss_fn = torch.nn.CrossEntropyLoss()
- torch.sum(torch.log(softmax(netout))*y_onehot)/5

`#`

`-` 계산하는 공식을 아는것도 중요한데 torch.nn.CrossEntropyLoss() 에는 softmax 활성화함수가 이미 포함되어 있다는 것을 확인하는 것이 더 중요함. 

`-` torch.nn.CrossEntropyLoss() 는 사실 torch.nn.CEWithSoftmaxLoss() 정도로 바꾸는 것이 더 말이 되는 것 같다. 

## E. Minor Topic: 이진분류와 CrossEntropy

`-` 2개의 클래스일경우에도 CrossEntropy를 쓸 수 있지 않을까? 

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data(fastai.data.external.URLs.MNIST)
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X = torch.concat([X0,X1]).reshape(-1,1*28*28)/255
y = torch.tensor([0]*len(X0) + [1]*len(X1))
## Step2: 학습가능한 오브젝트 생성
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Linear(784,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,2),
    #torch.nn.Softmax()
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
## Step3: fit  
for epoc in range(70): 
    ## 1 
    ## 2 
    loss= loss_fn(net(X),y) 
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad() 
## Step4: Predict 
softmax = torch.nn.Softmax()
(net(X).argmax(axis=1) == y).float().mean()

`-` 이진분류문제 = "y=0 or y=1" 을 맞추는 문제 = 성공과 실패를 맞추는 문제 = 성공확률과 실패확률을 추정하는 문제 

`-` softmax, sigmoid

- softmax: (실패확률, 성공확률) 꼴로 결과가 나옴 // softmax는 실패확률과 성공확률을 둘다 추정한다. 
- sigmoid: (성공확률) 꼴로 결과가 나옴 // sigmoid는 성공확률만 추정한다. 

`-` 그런데 "실패확률=1-성공확률" 이므로 사실상 둘은 같은걸 추정하는 셈이다. (성공확률만 추정하면 실패확률은 저절로 추정되니까) 

`-` 즉 아래는 같은 표현력을 가진 모형이다. 

![](https://raw.githubusercontent.com/guebin/DL2024/cdbdf23589efc2198260ab9c749f1757f67a128d/posts/05wk-1-fig2.svg)

![](https://raw.githubusercontent.com/guebin/DL2024/cdbdf23589efc2198260ab9c749f1757f67a128d/posts/05wk-1-fig1.svg)

`-` 둘은 같은 표현력을 가진 모형인데 학습할 파라메터는 sigmoid의 경우가 더 적다. $\to$ sigmoid를 사용하는 모형이 비용은 싸고 효과는 동일하다는 말 $\to$ 이진분류 한정해서는 softmax를 쓰지말고 sigmoid를 써야함. 

- softmax가 갑자기 너무 안좋아보이는데 sigmoid는 k개의 클래스로 확장이 불가능한 반면 softmax는 확장이 용이하다는 장점이 있음.

## F. 정리 

`-` 결론 

1. 소프트맥스는 시그모이드의 확장이다. 
2. 클래스의 수가 2개일 경우에는 (Sigmoid, BCEloss) 조합을 사용해야 하고 클래스의 수가 2개보다 클 경우에는 (Softmax, CrossEntropyLoss) 를 사용해야 한다. 


`-` 그런데 사실.. 클래스의 수가 2개일 경우일때 (Softmax, CrossEntropyLoss)를 사용해도 그렇게 큰일나는것은 아니다. (그냥 좀 비효율적인 느낌이 드는 것 뿐임. 흑백이미지를 칼라잉크로 출력하는 느낌) 

***참고***

|$y$|분포가정|마지막층의 활성화함수|손실함수|
|:--:|:--:|:--:|:--:|
|3.45, 4.43, ... (연속형) |정규분포|None (or Identity)|MSE|
|0 or 1|이항분포 with $n=1$ (=베르누이) |Sigmoid| BCE|
|[0,0,1], [0,1,0], [1,0,0]| 다항분포 with $n=1$|Softmax| Cross Entropy |

# 6. HW

아래와 같은 자료가 있다. 

In [ ]:
## Step1: 데이터준비 
path = fastai.data.external.untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
X0 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/0').ls()])
X1 = torch.stack([torchvision.io.read_image(str(fname)) for fname in (path/'training/1').ls()])
X = torch.concat([X0,X1]).reshape(-1,1*28*28)/255
y = torch.tensor([0]*len(X0) + [1]*len(X1))

`(1)` 세부사항에 맞추어 위의 자료를 학습하고, accuracy를 구하라. 

- 네트워크는 1개의 은닉층을 가지도록 하고, 은닉노드수는 32개로 설정하라. 은닉층의 활성화함수는 ReLU로 설정하라. 
- 손실함수를 `torch.nn.BCELoss()`로 설정하라.
- epoch = 325로 설정하라. 

`(2)` 세부사항에 맞추어 위의 자료를 학습하고, accuracy를 구하라. 

- 네트워크는 1개의 은닉층을 가지도록 하고, 은닉노드수는 32개로 설정하라. 은닉층의 활성화함수는 ReLU로 설정하라. 
- 손실함수를 `torch.nn.BCEWithLogitsLoss()`로 설정하라.
- epoch = 325로 설정하라. 

`(3)` 세부사항에 맞추어 위의 자료를 학습하고, accuracy를 구하라. 

- 네트워크는 1개의 은닉층을 가지도록 하고, 은닉노드수는 32개로 설정하라. 은닉층의 활성화함수는 ReLU로 설정하라. 
- y를 one_hot 인코딩하라. 
- 손실함수를 `torch.nn.CrossEntropyLoss()`로 설정하라.
- epoch = 325로 설정하라. 

**hint** 원핫인코딩을 위해 아래의 함수를 사용하라. 

In [ ]:
y, torch.nn.functional.one_hot(y)

`(4)` 세부사항에 맞추어 위의 자료를 학습하고, accuracy를 구하라. 

- 네트워크는 1개의 은닉층을 가지도록 하고, 은닉노드수는 32개로 설정하라. 은닉층의 활성화함수는 ReLU로 설정하라. 
- y를 (one-hot 인코딩 하지 않고) lenght-$n$인 벡터로 유지하라. 
- 손실함수를 `torch.nn.CrossEntropyLoss()`로 설정하라.
- epoch = 325로 설정하라. 

`(5)` 세부사항에 맞추어 위의 자료를 학습하고, accuracy를 구하라. 

- 네트워크는 1개의 은닉층을 가지도록 하고, 은닉노드수는 32개로 설정하라. 은닉층의 활성화함수는 ReLU로 설정하라. 
- batch_size = 1024 로 설정한뒤 mini_batch를 이용한 학습을 하라. 
- y를 (one-hot 인코딩 하지 않고) lenght-$n$인 벡터로 유지하라.  
- 손실함수를 `torch.nn.CrossEntropyLoss()`로 설정하라.
- 총 iteration 수가 325가 되도록 적절하게 epoch 을 설정하라. 

`(6)` 세부사항에 맞추어 위의 자료를 학습하고, accuracy를 구하라. 

- 네트워크는 1개의 은닉층을 가지도록 하고, 은닉노드수는 32개로 설정하라. 은닉층의 활성화함수는 ReLU로 설정하라. 
- batch_size = 512 로 설정한뒤 mini_batch를 이용한 학습을 하라. 
- y를 (one-hot 인코딩 하지 않고) lenght-$n$인 벡터로 유지하라. 
- 손실함수를 `torch.nn.CrossEntropyLoss()`로 설정하라.
- 총 iteration 수가 325가 되도록 적절하게 epoch 을 설정하라. 
- GPU를 활용하여 학습하라.